In [1]:
import pandas as pd
import numpy as np
import glob
import shutil
import os

In [2]:
def move_file(path,path_move,list_file):
    ''' 
    path= origin dir path where you have your files
    path_move= path dir  where you want to  move your files
    list_file= list of file that you want to move your files
    '''
    for f in list_file:
        shutil.copy(path+f, path_move)

def name(x):
    ''' 
    x=name of categorical (str)
    '''
    if x== 'coughing':
        return 'coughing'
    else:
        return 'no_coughing'


In [11]:
filename=esc50_labels[esc50_labels.category=='coughing']['filename']
move_file('../dataset/audio_ESC-50/',
          '../dataset/cough_esc50',
          list(filename))

In [3]:
esc50_labels = pd.read_csv("../dataset/esc50.csv")
category=['airplane','breathing','brushing_teeth','cat','crackling_fire','crying_baby','door_wood_creaks',
          'door_wood_knock','engine','glass_breaking','insects','pouring_water','rain','sneezing','snoring',
          'water_drops','wind','crickets','sea_waves']
cougth_negative=glob.glob('../dataset/cougth/negative/*.wav')
cougth_positive=glob.glob('../dataset/cougth/positive/*.wav')
cougth_bank=glob.glob('../dataset/cougth/tos_bank_cnv/*.wav')

In [4]:
####
tos_negative=pd.DataFrame()
tos_bank=pd.DataFrame()
tos_positive=pd.DataFrame()
####
tos_negative['filename']=[x.split('/')[4] for x in cougth_negative]
tos_negative['category']='coughing'
####
tos_positive['filename']=[x.split('/')[4] for x in cougth_positive]
tos_positive['category']='coughing'
####
tos_bank['filename']=[x.split('/')[4] for x in cougth_bank]
tos_bank['category']='coughing'
cough=pd.concat([tos_negative,tos_positive,tos_bank],axis=0)
####
#size_sample_test=15
#size_sample= len(cough) - size_sample_test
####
cough_sample_train=cough[cough.filename.isin(cough.filename.sample(150))]
cough_sample_train=(cough_sample_train.reset_index())[['filename', 'category']]
# Move file
move_file('../dataset/audio_ESC-50/',
          '../dataset/dataset_train_CNN_sigmo',
          list(cough_sample_train.filename))

In [7]:
### cough_sample_test
cough_sample_test=cough[np.logical_not(cough.filename.isin(cough_sample_train.filename))]
cough_sample_test=cough_sample_test.sample(15)
#cough_sample_test=cough[np.logical_not(cough.filename.isin(cough_sample_train.filename))]
move_file('../dataset/audio_ESC-50/',
          '../dataset/dataset_test_CNN_sigmo',
          list(cough_sample_test.filename))

##cough_sample_test
##cough_sample_train

In [ ]:
### no caugh

In [10]:
size_nocough=int(round(150/len(category),0))

In [11]:
#size_nocough=int(round(size_sample/len(category),0))
no_cough=esc50_labels[esc50_labels.category.isin(category)][['filename','category']]
no_cough_sample_train=no_cough.groupby('category')['filename'].apply(lambda s: s.sample(size_nocough))
no_cough_sample_train=(no_cough_sample_train.reset_index())[['filename', 'category']]

move_file('../dataset/audio_ESC-50/',
          '../dataset/dataset_train_CNN_sigmo',
          list(no_cough_sample_train.filename))


In [12]:
no_cough_nosample=no_cough[np.logical_not(no_cough.filename.isin(no_cough_sample_train))]
size_nocough_test=int(round(len(cough_sample_test)/len(category),0))
no_cough_sample_test=no_cough_nosample.groupby('category')['filename'].apply(lambda s: s.sample(size_nocough_test))
no_cough_sample_test=(no_cough_sample_test.reset_index())[['filename', 'category']]

move_file('../dataset/audio_ESC-50/',
          '../dataset/dataset_test_CNN_sigmo',
          list(no_cough_sample_test.filename))
### no_cough_sample_train
### no_cough_sample_test

In [13]:
### csv train
csv_train_data=cough_sample_train.append(no_cough_sample_train)
csv_train_data['target']=csv_train_data.category.apply(name)
csv_train_data.to_csv('../metadata/dataset_train_cnn.csv',index=False)
### csv test
csv_test_data=no_cough_sample_test.append(cough_sample_test)
csv_test_data['target']=csv_test_data.category.apply(name)
csv_test_data.to_csv('../metadata/dataset_test_cnn.csv',index=False)